# Lecture01 O/S Overivew

## Overview

**Course goals**

* Understand operating systems in detail by designing and implementing a small O/S
* Hands-on experience with building systems


**What do applications want from an O/S?**

* Abstract the hardware for convenience and portability
* Multiplex the hardware among multiple applications
* Isolate applications to contain bugs
* Allow sharing among applications


**What is an OS?**

* the small view: a hardware management library
* the big view: physical machine -> abstract one w/ better properties


**What services does an O/S kernel typically provide?**

* processes
* memory
* file contents
* directories and file names
* security
* many others: users, IPC, network, time, terminals

## Class Structure

See [web site](https://pdos.csail.mit.edu/6.828).

**Lectures**

* basic O/S ideas
* extended inspection of xv6, a traditional O/S
* several more recent topics
* xv6 programming to re-inforce xv6 understanding

**Labs**: JOS, a small O/S for x86 in an exokernel style

* 5 labs + final lab
* unprivileged library: fork, exec, pipe, ...
* applications: file system, shell, ...
* development environment: gcc, qemu

## Example system calls

**6.828** is largely about design and implementation of system call interface.

**A simple example**:

what system calls does "ls" call? Trace system calls:

* On OSX: `sudo dtruss /bin/ls`
* On Linux: `strace /bin/ls`

Simpler program, `echo`: 

``` C
#include <unistd.h>
#define BUFSIZE 128

char buf[BUFSIZE];
int main(void) {
  int n = read(0, buf, BUFSIZE);
  write (1, buf, n);
}
```

Redirect I/O to a file:

``` C
#include <stdio.h>
#define BUFSIZE 128

char buf[BUFSIZE];
int main(void) {
  int n = read(0, buf, BUFSIZE);
  close(1);
  int fd = creat("output.txt", 0666);
  fprintf(stderr, "fd = %d\n", fd);
  write (fd, buf, n);
}
```

**The Unix shell**

* the Unix command UI
* handles login session, runs other processes
* a programming/scripting language

Some simple examples of shell operations, [Unix paper](https://pdos.csail.mit.edu/6.828/2016/readings/ritchie78unix.pdf).



## Homework

Read [xv6-book Chapter 0](http://nbviewer.jupyter.org/github/JeraKrs/Notes/blob/master/Operating%20System/xv6-book/chapter0.ipynb).

**Executing simple commands**

``` C
  case ' ':
    ecmd = (struct execcmd*)cmd;
    if(ecmd->argv[0] == 0)
      exit(0);
    execv(ecmd->argv[0], ecmd->argv);
    fprintf(stderr, "exec not implemented\n");
    break;
```

**I/O redirection**

``` C
  case '>':
    rcmd = (struct redircmd*)cmd;
	close(1);
	if (open(rcmd->file, O_CREAT | O_WRONLY | O_TRUNC, 0666)) {
		fprintf(stderr, "open %s fail.\n", rcmd->file);
	}
    runcmd(rcmd->cmd);
    break;

  case '<':
    rcmd = (struct redircmd*)cmd;
	close(0);
	if (open(rcmd->file, O_RDONLY, 0666) < 0) {
		fprintf(stderr, "open %s fail.\n", rcmd->file);
	}
    runcmd(rcmd->cmd);
    break;
```

**Implement pipes**

``` C
  case '|':
    pcmd = (struct pipecmd*)cmd;

	if (pipe(p) < 0) {
		fprintf(stderr, "pipe is failed to create.\n");
	}

	if (fork1() == 0) {
		close(1);
		dup(p[1]);
		close(p[0]);
		close(p[1]);
		runcmd(pcmd->left);
	}
	
	if (fork1() == 0) {
		close(0);
		dup(p[0]);
		close(p[0]);
		close(p[1]);
		runcmd(pcmd->right);
	}
	close(p[0]);
	close(p[1]);
	wait(NULL);
	wait(NULL);
    break;
```